# Parent Document Retriever
**문서 검색과 문서 분할의 균형 잡기**

1. 문서 검색 과정에서 문서를 적절한 크기의 조각(청크)으로 나누는 것은 다음의 상충되는 두 가지 중요한 요소를 고려해야 합니다.

2. 작은 문서를 원하는 경우: 이렇게 하면 문서의 임베딩이 그 의미를 가장 정확하게 반영할 수 있습니다. 문서가 너무 길면 임베딩이 의미를 잃어버릴 수 있습니다.
각 청크의 맥락이 유지되도록 충분히 긴 문서를 원하는 경우입니다.

**ParentDocumentRetriever의 역할**

이 두 요구 사항 사이의 균형을 맞추기 위해 `ParentDocumentRetriever`라는 도구가 사용됩니다. 이 도구는 문서를 작은 조각으로 나누고, 이 조각들을 관리합니다. 검색을 진행할 때는, 먼저 이 작은 조각들을 찾아낸 다음, 이 조각들이 속한 원본 문서(또는 더 큰 조각)의 식별자(ID)를 통해 전체적인 맥락을 파악할 수 있습니다.

여기서 '부모 문서'란, 작은 조각이 나누어진 원본 문서를 말합니다. 이는 전체 문서일 수도 있고, 비교적 큰 다른 조각일 수도 있습니다. 이 방식을 통해 문서의 의미를 정확하게 파악하면서도, 전체적인 맥락을 유지할 수 있게 됩니다.

**정리**

- `문서 간의 계층 구조 활용`: ParentDocumentRetriever는 문서 검색의 효율성을 높이기 위해 문서 간의 계층 구조를 활용합니다.
- `검색 성능 향상`: 관련성 높은 문서를 빠르게 찾아내며, 주어진 질문에 대한 가장 적합한 답변을 제공하는 문서를 효과적으로 찾아낼 수 있습니다. 문서를 검색할 때 자주 발생하는 두 가지 상충되는 요구 사항이 있습니다:
여러 개의 텍스트 파일을 로드하기 위해 TextLoader 객체를 생성하고 데이터를 로드합니다.

- child_splitter로 분할된 문서를 찾으면, parent_splitter로 분할된 문서를 반환한다고 생각하면 된다.
- parentanst는 InMemoryStore()를 통해 docstore형태로 저장한다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [4]:
loaders = [
    TextLoader("./data/appendix-keywords.txt")
]
docs = []

for loader in loaders:
    docs.extend(loader.load())

In [6]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

child_splitter = RecursiveCharacterTextSplitter(chunk_size=300)

vector_store = Chroma(collection_name="full_documents", embedding_function=embedding)

store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=store, # 원본 문서를 저장하기 위한 용도
    child_splitter=child_splitter
)

`retriever.add_documents(docs, ids=None)` 함수로 문서목록을 추가합니다.

- `ids` 가 None 이면 자동으로 생성됩니다.
- `add_to_docstore=False` 로 설정시 document 를 중복으로 추가하지 않습니다. 단, 중복을 체크하기 위한 ids 값이 필수 값으로 요구됩니다

In [7]:
retriever.add_documents(docs, ids=None, add_to_docstore=True)

이 코드는 두 개의 키를 반환해야 합니다. 그 이유는 우리가 두 개의 문서를 추가했기 때문입니다.

`store` 객체의 yield_keys() 메서드를 호출하여 반환된 키(key) 값들을 리스트로 변환합니다.

In [11]:
# 저장소의 모든 키를 리스트로 반환합니다.
list(store.yield_keys())

['4dada15b-4e56-43de-944b-f2fe5412e092']

이제 벡터 스토어 검색 기능을 호출해 보겠습니다.

우리가 작은 청크(chunk)들을 저장하고 있기 때문에, 검색 결과로 작은 청크들이 반환되는 것을 확인할 수 있을 것입니다.

vectorstore 객체의 similarity_search 메서드를 사용하여 유사도 검색을 수행합니다.

In [8]:
sub_docs = vector_store.similarity_search("Word2Vec")

In [9]:
print(sub_docs[0].page_content)

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


In [12]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Word2Vec")

In [21]:
print(len(retrieved_docs))
print(retrieved_docs[0].page_content[0:100])

1
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태


# 더 큰 Chunk 의 크기를 조절
이전의 결과처럼 전체 문서가 너무 커서 있는 그대로 검색하기에는 부적합 할 수 있습니다.

이런 경우, 실제로 우리가 하고 싶은 것은 먼저 원시 문서를 더 큰 청크로 분할한 다음, 더 작은 청크로 분할하는 것입니다.

그런 다음 작은 청크들을 인덱싱하지만, 검색 시에는 더 큰 청크를 검색합니다 (그러나 여전히 전체 문서는 아닙니다).

- RecursiveCharacterTextSplitter를 사용하여 부모 문서와 자식 문서를 생성합니다.
- 부모 문서는 chunk_size가 1000으로 설정되어 있습니다.
- 자식 문서는 chunk_size가 200으로 설정되어 있으며, 부모 문서보다 작은 크기로 생성됩니다.

In [22]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

vector_store = Chroma(
    collection_name="split_parents", embedding_function=embedding
)
store = InMemoryStore()

### ParentDocumentRetriever를 초기화하는 코드입니다.

- `vectorstore` 매개변수는 문서 벡터를 저장하는 벡터 저장소를 지정합니다.
- `docstore` 매개변수는 문서 데이터를 저장하는 문서 저장소를 지정합니다.
- `child_splitter` 매개변수는 하위 문서를 분할하는 데 사용되는 문서 분할기를 지정합니다.
- `parent_splitter` 매개변수는 상위 문서를 분할하는 데 사용되는 문서 분할기를 지정합

In [23]:
retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

In [24]:
retriever.add_documents(docs)

In [28]:
# 저장소에서 키를 생성하고 리스트로 변환한 후 길이를 반환합니다.
print(len(list(store.yield_keys())))
print(list(store.yield_keys()))

7
['75671e61-37f7-470f-8493-5b7932333d86', 'bd373aee-7a44-45a6-adac-a2a7ca7d79e4', '41185782-2193-425b-901e-4b426fe6d850', '21878abc-de83-495f-b4c9-9acbc49796d7', '79f61c1b-b24b-4c52-b0f0-df572ec1dbff', '45fa6d49-afaf-4881-90f5-471705b617cc', 'c1f86bff-6f0e-46c3-8c3c-3de9f06eae5a']


In [29]:
sub_docs = vector_store.similarity_search("Word2Vec")
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


In [31]:
retrieved_docs = retriever.invoke("Word2Vec")
print(retrieved_docs[0].page_content)

정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.
예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.
연관키워드: 딥러닝, 자연어 처리, Attention

HuggingFace

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
